In [1]:
import sys
import numpy
from matplotlib import pyplot
from tensorflow import keras
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.losses import BinaryCrossentropy
# from keras.metrics import Accuracy
from tensorflow.keras.metrics import BinaryAccuracy
# from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import io

logdir = "logs"
file_writer = tf.summary.create_file_writer(logdir)

def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image

def image_grid():
    """Return a 4x5 grid of the MNIST images as a matplotlib figure."""
    # Create a figure to contain the plot.
    figure = plt.figure(figsize=(10,10))
    for i in range(20):
        test=np.reshape(test_it[0][0][i], (-1, 224, 224, 3))
        if model.predict(test)>0.5:
            tle='squirrel'
        else:
            tle='rabbit'
        plt.subplot(4, 5, i + 1, title=tle)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow((test_it[0][0][i])/100)

# create data generator
datagen = ImageDataGenerator(featurewise_center=True)
# specify imagenet mean values for centering
datagen.mean = [123.68, 116.779, 103.939]
# prepare iterator
train_it = datagen.flow_from_directory('images/train/',
class_mode='binary', batch_size=64, target_size=(224, 224))
test_it = datagen.flow_from_directory('images/test/',
class_mode='binary', batch_size=64, target_size=(224, 224))

def define_model():
    # load model
    model = VGG16(include_top=False, input_shape=(224, 224, 3))
    # mark loaded layers as not trainable
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    # # compile model
    # opt = SGD(learning_rate=0.001, momentum=0.9)
    # model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model


model = define_model()
num_epochs = 10
binary_crossentropy = BinaryCrossentropy()
optimizer = SGD(learning_rate=0.001, momentum=0.9)
# accuracy_metric = Accuracy()
accuracy_metric = BinaryAccuracy()
train_writer = tf.summary.create_file_writer("logs/train")
test_writer = tf.summary.create_file_writer("logs/test")
train_step = test_step = 0

batch_size=64
num_train_images=160 # for both classes
num_test_images=40 # for both classes

train_loop_iter=numpy.ceil(num_train_images/batch_size)
test_loop_iter=numpy.ceil(num_test_images/batch_size)

for epoch in range(num_epochs):
    # Iterate through training set

    for batch_idx, (x, y) in enumerate(train_it):
        # print(f"Batch {batch_idx}: x shape = {x.shape}, y shape = {y.shape}")

        with tf.GradientTape() as tape:
            y_pred = model(x, training=True)
            loss = binary_crossentropy(y, y_pred)

        gradients = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(gradients, model.trainable_weights))
        accuracy_metric.update_state(y, y_pred)

        with train_writer.as_default():
            tf.summary.scalar("Train Loss", loss, step=train_step)
            tf.summary.scalar(
                "Train Accuracy", accuracy_metric.result().numpy(), step=train_step,
            )
            train_step += 1

        train_loop_iter=train_loop_iter-1
        if train_loop_iter==0:
            train_loop_iter=numpy.ceil(num_train_images/batch_size)
            break
    train_writer.flush()
    print(f"Train Loss (Epoch {epoch + 1}): {loss.numpy():.4f}")
    print(f"Train Accuracy (Epoch {epoch + 1}): {accuracy_metric.result().numpy()*100:.4f}%")

    # Reset accuracy in between epochs (and for testing and test)
    accuracy_metric.reset_state()

    # Iterate through test set
    for batch_idx, (x, y) in enumerate(test_it):
        y_pred = model(x, training=False)
        loss = binary_crossentropy(y, y_pred)
        accuracy_metric.update_state(y, y_pred)

        with test_writer.as_default():
            tf.summary.scalar("Test Loss", loss, step=test_step)
            tf.summary.scalar(
                "Test Accuracy", accuracy_metric.result().numpy(), step=test_step,
            )
            test_step += 1

        test_loop_iter=test_loop_iter-1
        if test_loop_iter==0:
            test_loop_iter=numpy.ceil(num_test_images/batch_size)
            break
    test_writer.flush()
    print(f"Test Loss (Epoch {epoch + 1}): {loss.numpy():.4f}")
    print(f"Test Accuracy (Epoch {epoch + 1}): {accuracy_metric.result().numpy()*100:.4f}%")

    accuracy_metric.reset_state()

# Prepare the plot
figure = image_grid()
# Convert to image and log
with file_writer.as_default():
    tf.summary.image("Test data", plot_to_image(figure), step=0)
file_writer.flush()

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


C:\Users\asus\AppData\Roaming\Python\Python311\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Train Loss (Epoch 1): 3.5034
Train Accuracy (Epoch 1): 55.6250%
Test Loss (Epoch 1): 0.3180
Test Accuracy (Epoch 1): 87.5000%
Train Loss (Epoch 2): 0.3213
Train Accuracy (Epoch 2): 89.3750%
Test Loss (Epoch 2): 0.0491
Test Accuracy (Epoch 2): 97.5000%
Train Loss (Epoch 3): 0.0052
Train Accuracy (Epoch 3): 96.8750%
Test Loss (Epoch 3): 0.0044
Test Accuracy (Epoch 3): 100.0000%
Train Loss (Epoch 4): 0.4030
Train Accuracy (Epoch 4): 98.7500%
Test Loss (Epoch 4): 0.0085
Test Accuracy (Epoch 4): 100.0000%
Train Loss (Epoch 5): 0.0001
Train Accuracy (Epoch 5): 99.3750%
Test Loss (Epoch 5): 0.0025
Test Accuracy (Epoch 5): 100.0000%
Train Loss (Epoch 6): 0.0002
Train Accuracy (Epoch 6): 100.0000%
Test Loss (Epoch 6): 0.0015
Test Accuracy (Epoch 6): 100.0000%
Train Loss (Epoch 7): 0.0004
Train Accuracy (Epoch 7): 100.0000%
Test Loss (Epoch 7): 0.0010
Test Accuracy (Epoch 7): 100.0000%
Train Loss (Epoch 8): 0.0001
Train Accuracy (Epoch 8): 100.0000%
Test Loss (Epoch 8): 0.0007
Test Accuracy (Epo

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step


...

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [2]:
%load_ext tensorboard
%tensorboard --logdir logs


Reusing TensorBoard on port 6006 (pid 19424), started 13:07:15 ago. (Use '!kill 19424' to kill it.)